In [9]:
import numpy as np
from CLEAR_Interface import RUN_CLEAR as CLEAR
from int_met import * #(from:  https://github.com/Mo-youssef/interpretability/blob/master/int_met.py)


In [10]:
n_obs_test=2
results1,time1,test_data1=CLEAR('BreastC',1,n_obs_test)
#results2,time2,test_data2=CLEAR.run_Clear('BreastC',1,n_obs_test)

Pre-processing 

Accuracy statistics for Random Forest to be explained 

0.9534883720930233

 Performing grid search - step 1 of CLEAR method 

Performing step-wise regressions 

Processed observation 1
Pre-processing 

Accuracy statistics for Random Forest to be explained 

0.9534883720930233

 Performing grid search - step 1 of CLEAR method 

Performing step-wise regressions 

Processed observation 2


In [11]:
def get_coefs(results):
    weights=[]
    for result in results:
        weights = weights + [np.insert(results1[0].weights.tolist()[0],0,result.intercept.tolist()[0]).tolist()]
    return weights

print(np.shape(get_coefs(results1)))

(2, 15)


In [12]:
y_test=np.asarray(test_data1[1].tolist()[0:n_obs_test])
X_test=test_data1[0][0:n_obs_test]
coefs1 = get_coefs(results1)
coefs1 = np.array(coefs1)
#coefs2 = get_coefs(results2)
#coefs2=[float(results2.intercept)]+results2.weights.tolist()[0].tolist()

In [13]:
identity_score = calc_identity(coefs1, coefs2)
print("Identity Metric Scores for CLEAR: ", identity_score)

NameError: name 'coefs2' is not defined

In [22]:
print(calc_stability2(coefs1, y_test))

labels [0 1]
label_values [0 1]
exp [[ 0.4607293  -0.0354308   0.00676099  0.25735907 -0.04399231  0.0971242
   0.1125065   0.02294014  0.0819772   0.20133953  0.03945048 -0.11132151
  -0.06482894  0.04656462  0.03848108]
 [ 0.3483515  -0.0354308   0.00676099  0.25735907 -0.04399231  0.0971242
   0.1125065   0.02294014  0.0819772   0.20133953  0.03945048 -0.11132151
  -0.06482894  0.04656462  0.03848108]]


/home/abdul/anaconda3/envs/BDA/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  # This is added back by InteractiveShellApp.init_path()


(0, 2)


In [19]:
print(calc_separability(X_test,coefs1))

(2, 0, 100.0)


In [26]:
coefs1

array([[ 0.4607293 , -0.0354308 ,  0.00676099,  0.25735907, -0.04399231,
         0.0971242 ,  0.1125065 ,  0.02294014,  0.0819772 ,  0.20133953,
         0.03945048, -0.11132151, -0.06482894,  0.04656462,  0.03848108],
       [ 0.3483515 , -0.0354308 ,  0.00676099,  0.25735907, -0.04399231,
         0.0971242 ,  0.1125065 ,  0.02294014,  0.0819772 ,  0.20133953,
         0.03945048, -0.11132151, -0.06482894,  0.04656462,  0.03848108]])

In [14]:
y_test

array([0, 1])

In [21]:
def calc_stability2(exp, labels):
    total = labels.shape[0]
    label_values = np.unique(labels)
    print("labels", labels)

    print("label_values", label_values)
    print("exp", exp)
    n_clusters = label_values.shape[0]
    init = np.array([[np.average(exp[np.where(labels == i)], axis = 0)] for i in label_values]).squeeze()
    ct = sklearn.cluster.KMeans(n_clusters = n_clusters, n_jobs=5, random_state=1, n_init=10, init = init)
    ct.fit(exp)
    error = np.sum(np.abs(labels-ct.labels_))
    if error/total > 0.5:
        error = total-error
    return error, total